<a href="https://colab.research.google.com/github/Chavamanish/Intrution-detection-system-IDS-/blob/main/NIDS_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('/content/drive/MyDrive/NIDS DATASET/NSLKDD_TRAIN.csv')
test_df = pd.read_csv('/content/drive/MyDrive/NIDS DATASET/NSLKDD_Test.csv')


In [ ]:
train_df['protocol_type'].value_counts()

tcp     102688
udp      14993
icmp      8291
Name: protocol_type, dtype: int64

In [ ]:
np.unique(train_df['protocol_type'])

array(['icmp', 'tcp', 'udp'], dtype=object)

In [ ]:
np.unique(train_df['service'])

array(['IRC', 'X11', 'Z39_50', 'aol', 'auth', 'bgp', 'courier',
       'csnet_ns', 'ctf', 'daytime', 'discard', 'domain', 'domain_u',
       'echo', 'eco_i', 'ecr_i', 'efs', 'exec', 'finger', 'ftp',
       'ftp_data', 'gopher', 'harvest', 'hostnames', 'http', 'http_2784',
       'http_443', 'http_8001', 'imap4', 'iso_tsap', 'klogin', 'kshell',
       'ldap', 'link', 'login', 'mtp', 'name', 'netbios_dgm',
       'netbios_ns', 'netbios_ssn', 'netstat', 'nnsp', 'nntp', 'ntp_u',
       'other', 'pm_dump', 'pop_2', 'pop_3', 'printer', 'private',
       'red_i', 'remote_job', 'rje', 'shell', 'smtp', 'sql_net', 'ssh',
       'sunrpc', 'supdup', 'systat', 'telnet', 'tftp_u', 'tim_i', 'time',
       'urh_i', 'urp_i', 'uucp', 'uucp_path', 'vmnet', 'whois'],
      dtype=object)

In [ ]:
np.unique(train_df['flag'])

array(['OTH', 'REJ', 'RSTO', 'RSTOS0', 'RSTR', 'S0', 'S1', 'S2', 'S3',
       'SF', 'SH'], dtype=object)

In [ ]:
attack_types = {
    "back":"DOS",
    "land":"DOS",
    "neptune":"DOS",
    "pod":"DOS",
    "smurf":"DOS",
    "teardrop":"DOS",
    "apache2":"DOS",
    "udpstorm":"DOS",
    "processtable":"DOS",
    "worm":"DOS",
    "satan":"PROBE",
    "ipsweep":"PROBE",
    "nmap":"PROBE",
    "portsweep":"PROBE",
    "mscan":"PROBE",
    "saint":"PROBE",
    "guess_passwd":"R2L",
    "ftp_write":"R2L",
    "imap":"R2L",
    "phf":"R2L",
    "multihop":"R2L",
    "warezmaster":"R2L",
    "warezclient":"R2L",
    "spy":"R2L",
    "xlock":"R2L",
    "xsnoop":"R2L",
    "snmpguess":"R2L",
    "snmpgetattack":"R2L",
    "httptunnel":"R2L",
    "sendmail":"R2L",
    "named":"R2L",
    "buffer_overflow":"U2R",
    "loadmodule":"U2R",
    "rootkit":"U2R",
    "perl":"U2R",
    "sqlattack":"U2R",
    "xterm":"U2R",
    "ps":"U2R",
    "mailbomb":"DOS",
    "normal":"NORMAL"
    
}

In [ ]:
train_df['attack'] = train_df['attack'].map(attack_types)
test_df['attack'] = test_df['attack'].map(attack_types)

In [ ]:
train_15_df = pd.read_csv('/content/drive/MyDrive/NIDS DATASET/NSLKDD_top_15_TRAIN.csv')
test_15_df = pd.read_csv('/content/drive/MyDrive/NIDS DATASET/NSLKDD_top_15_TEST.csv')

In [ ]:
x_train_15 = train_15_df.iloc[:,0:-1].values
y_train_15= train_15_df.iloc[:,-1].values
x_test_15 = test_15_df.iloc[:,0:-1].values
y_test_15 = test_15_df.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_15 = sc.fit_transform(x_train_15)
x_test_15 = sc.transform(x_test_15)

In [ ]:
import tensorflow as tf
def build_model_15(hp):
    ann_15 = tf.keras.Sequential()

    ann_15.add(
        tf.keras.layers.Dense(
            units=15,
            activation=hp.Choice(f'Input layer activation func',['relu','tanh'])
            ))
    
    for i in range(hp.Int("DENSE_LAYER",1,10)):
      ann_15.add(tf.keras.layers.Dense(units=hp.Int(f'Layer{i}',min_value=5,max_value=15,step=2),activation=hp.Choice(f'Layer{i} activation',['relu','tanh'])))
    ann_15.add(tf.keras.layers.Dense(units=5, activation='softmax'))
    ann_15.compile(
        optimizer="adam", loss="SparseCategoricalCrossentropy", metrics=["accuracy"],
    )
    return ann_15


In [ ]:
!pip install keras-tuner -q

     |████████████████████████████████| 133 kB 14.8 MB/s 


In [ ]:
 from keras_tuner import RandomSearch
 from keras_tuner.engine.hyperparameters import HyperParameters
 

In [ ]:
tuner_search=RandomSearch(hypermodel=build_model_15,objective="val_accuracy",max_trials=3,directory="nids_dir",project_name="NIDSCLF")


In [ ]:
tuner_search.search(x_train_15,y_train_15,epochs=3,validation_data=(x_test_15,y_test_15))

Trial 3 Complete [00h 00m 20s]
val_accuracy: 0.8045069575309753

Best val_accuracy So Far: 0.8045069575309753
Total elapsed time: 00h 01m 09s
INFO:tensorflow:Oracle triggered exit


In [ ]:
ann_15=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
ann_15.fit(x_train_15,y_train_15,validation_data=(x_test_15,y_test_15),epochs=5,initial_epoch=3)

Epoch 4/5
3937/3937 [==============================] - 13s 3ms/step - loss: 0.0248 - accuracy: 0.9932 - val_loss: 1.1876 - val_accuracy: 0.8027
Epoch 5/5
3937/3937 [==============================] - 9s 2ms/step - loss: 0.0240 - accuracy: 0.9933 - val_loss: 1.1722 - val_accuracy: 0.8050


In [ ]:
train_df = train_df.drop(columns=['service','last_flag'],axis=1)

In [ ]:
test_df = test_df.drop(columns=['service','last_flag'],axis=1)

In [ ]:
x_train = train_df.iloc[:,0:-1].values
y_train= train_df.iloc[:,-1].values
x_test = test_df.iloc[:,0:-1].values
y_test = test_df.iloc[:,-1].values

In [ ]:
x_train

array([[0, 'udp', 'SF', ..., 0.0, 0.0, 0.0],
       [0, 'tcp', 'S0', ..., 1.0, 0.0, 0.0],
       [0, 'tcp', 'SF', ..., 0.01, 0.0, 1.0],
       ...,
       [0, 'tcp', 'SF', ..., 0.0, 0.01, 0.0],
       [0, 'tcp', 'S0', ..., 1.0, 0.0, 0.0],
       [0, 'tcp', 'SF', ..., 0.0, 0.0, 0.0]], dtype=object)

In [ ]:
y_train

array(['NORMAL', 'DOS', 'NORMAL', ..., 'NORMAL', 'DOS', 'NORMAL'],
      dtype=object)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,2])], remainder='passthrough')
x_train = ct.fit_transform(x_train)
x_test = ct.fit_transform(x_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train[:] = le.fit_transform(y_train[:])
y_test[:] = le.fit_transform(y_test[:])

In [ ]:
y_train

array([1, 0, 1, ..., 1, 0, 1], dtype=object)

In [ ]:
le.classes_

array(['DOS', 'NORMAL', 'PROBE', 'R2L', 'U2R'], dtype=object)

In [ ]:
x_train.shape

(125972, 52)

In [ ]:
y_train.shape

(125972,)

In [ ]:
def build_model_52(hp):
    ann_52 = tf.keras.Sequential()

    ann_52.add(
        tf.keras.layers.Dense(
            units=52,
            activation=hp.Choice(f'Input layer activation func',['relu','tanh'])
            ))
    
    for i in range(hp.Int("DENSE_LAYER",1,10)):
      ann_52.add(tf.keras.layers.Dense(units=hp.Int(f'Layer{i}',min_value=5,max_value=52,step=2),activation=hp.Choice(f'Layer{i} activation',['relu','tanh'])))
    ann_52.add(tf.keras.layers.Dense(units=5, activation='softmax'))
    ann_52.compile(
        optimizer="adam", loss="SparseCategoricalCrossentropy", metrics=["accuracy"],
    )
    return ann_52

In [ ]:
tuner_search_52 = RandomSearch(hypermodel=build_model_52,objective='val_accuracy',max_trials=3,directory="nids_dir_52",project_name="NIDSCLF_52")

In [ ]:
type(x_train)

numpy.ndarray

In [ ]:
tf_tensor_x_train = tf.convert_to_tensor(x_train,dtype=tf.float32)

In [ ]:
tf_tensor_x_test = tf.convert_to_tensor(x_test,dtype=tf.float32)

In [ ]:
tf_tensor_y_train = tf.convert_to_tensor(y_train,dtype=tf.float32)
tf_tensor_y_test = tf.convert_to_tensor(y_test,dtype=tf.float32)

In [ ]:
tuner_search_52.search(tf_tensor_x_train,tf_tensor_y_train,epochs=3,validation_data=(tf_tensor_x_test,tf_tensor_y_test))

Trial 3 Complete [00h 00m 41s]
val_accuracy: 0.7176950573921204

Best val_accuracy So Far: 0.7313134670257568
Total elapsed time: 00h 19m 01s
INFO:tensorflow:Oracle triggered exit


In [ ]:
ann_52=tuner_search_52.get_best_models(num_models=1)[0]

In [ ]:
ann_52.fit(tf_tensor_x_train,tf_tensor_y_train,validation_data=(tf_tensor_x_test,tf_tensor_y_test),epochs=6,initial_epoch=3)

Epoch 4/6
3937/3937 [==============================] - 8s 2ms/step - loss: 0.0457 - accuracy: 0.9870 - val_loss: 1.8951 - val_accuracy: 0.7352
Epoch 5/6
3937/3937 [==============================] - 8s 2ms/step - loss: 0.0445 - accuracy: 0.9864 - val_loss: 2.1623 - val_accuracy: 0.7031
Epoch 6/6
3937/3937 [==============================] - 7s 2ms/step - loss: 0.0475 - accuracy: 0.9864 - val_loss: 1.8355 - val_accuracy: 0.7326


In [ ]:
ann_52.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 52)                2756      
                                                                 
 dense_1 (Dense)             (None, 49)                2597      
                                                                 
 dense_2 (Dense)             (None, 5)                 250       
                                                                 
 dense_3 (Dense)             (None, 5)                 30        
                                                                 
 dense_4 (Dense)             (None, 5)                 30        
                                                                 
 dense_5 (Dense)             (None, 5)                 30        
                                                                 
Total params: 5,693
Trainable params: 5,693
Non-trainabl

In [ ]:
ann_15.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                240       
                                                                 
 dense_1 (Dense)             (None, 5)                 80        
                                                                 
 dense_2 (Dense)             (None, 7)                 42        
                                                                 
 dense_3 (Dense)             (None, 5)                 40        
                                                                 
Total params: 402
Trainable params: 402
Non-trainable params: 0
_________________________________________________________________
